***
**<center><font size = "6"> Part 1 - Risk Model Challenge Analysis</font><center>**
***
<center><font size = "2">Prepared by: Nahim Olguin<center>

# Table of Contents
1. [Business Objective](#Business-Objective)
2. [Exploratory Data Analysis (EDA)](#Exploratory-Data-Analysis-(EDA))

    - [Univariate Analysis](#Univariate-Analysis)
    - [Multivariate Analysis](#Multivariate-Analysis)
    - [Data Visualization](#Data-Visualization)
    
3. [Data Modeling](#Data-Modeling)
    - [Data Preprocessing](#Data-Preprocessing)
    - [Data Cleaning](#Data-Cleaning)
    - [Data Transformation](#Data-Transformation-Strategy)
    - [Encoding](#Encoding)
    - [Feature Selection](#Feature-Selection)
    - [Model Selection](#Model-Selection-Train-&-Test)
4. [Model Evaluation](#Model-Evaluation)
    - [Cross-validation](#Cross-validation)
    - [Metrics](#Metrics)
5. [Web Integration](#Web-Integration)

In [26]:
%%html
<style>
    .qst {padding:10px; border-radius: 5px; border: solid 2px #EF6E1A;}
    .qst:before {font-weight: bold;display: block; margin: 0px 10px 10px 10px;}
    h1,h3 {color: #EF6E1A;}
</style>

## Business Objective


To develop a predictive model that effectively differentiates between good and risky customers for Bankaya, thereby aiding in more accurate credit approval decisions. The model will specifically focus on new customers applying for their first loan to purchase a smartphone. Each step in the development process must be rigorously justified and documented to ensure both efficacy and transparency. 

You can find the repository in Github in the following link: <https://archive.ics.uci.edu/ml/datasets/Online+Retail#>

***

In [4]:
# -- Import necessary libraries
import pandas as pd

In [17]:
customer_loan_df = pd.read_parquet("input/main_dataset.parquet") #---> main dataset
credit_report_df = pd.read_parquet("input/credit_reports.parquet") # ---> credit reports


In [18]:
customer_loan_df.head(10)

,customer_id,loan_id,ACC_CREATION_DATETIME,APPLICATION_DATETIME,LOAN_ORIGINATION_DATETIME,max_days_late,target,account_to_application_days,n_sf_apps,first_app_date,last_app_date,n_bnpl_apps,n_bnpl_approved_apps,first_bnpl_app_date,last_bnpl_app_date,n_inquiries_l3m,n_inquiries_l6m
0,1223,1,2021-08-23 13:57:56,2022-04-26 07:00:00,2022-10-20 18:26:35,5,0,245,1.0,2022-04-25,2022-04-25,1.0,0.0,2022-04-26 02:15:17.742,2022-04-26 02:15:17.742,91.0,91.0
1,5190,2,2022-04-26 09:57:25,2022-04-26 07:00:00,2022-12-22 18:03:32,2,0,0,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN
2,5194,3,2022-04-26 12:22:35,2022-04-26 07:00:00,2022-08-09 17:31:05,4,0,0,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN
3,3978,4,2022-03-09 11:26:55,2022-04-26 07:00:00,2022-11-11 19:48:08,2,0,47,1.0,2022-03-17,2022-03-17,2.0,0.0,2022-03-19 22:02:50.555,2022-03-09 16:41:49.675,13.0,13.0
4,4535,5,2022-04-01 14:28:42,2022-04-26 07:00:00,2022-07-24 12:07:15,2,0,24,1.0,2022-04-01,2022-04-01,1.0,0.0,2022-04-01 19:37:56.185,2022-04-01 19:37:56.185,21.0,21.0
5,3604,6,2022-02-21 11:55:32,2022-05-05 07:00:00,2022-12-08 07:12:18,7,0,72,1.0,2022-02-21,2022-02-21,NaN,NaN,NaT,NaT,26.0,26.0
6,271,7,2021-04-06 16:30:21,2022-05-05 07:00:00,2023-04-21 15:40:10,0,0,393,NaN,NaT,NaT,1.0,0.0,2022-05-04 23:18:03.604,2022-05-04 23:18:03.604,11.0,11.0
7,5430,8,2022-05-05 18:28:18,2022-05-05 07:00:00,2023-01-10 20:02:22,0,0,0,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN
8,5128,9,2022-04-22 17:53:08,2022-05-05 07:00:00,2022-09-30 14:31:03,7,0,12,2.0,2022-05-05,2022-04-22,1.0,0.0,2022-04-22 22:59:05.990,2022-04-22 22:59:05.990,8.0,8.0
9,4402,10,2022-03-26 18:39:59,2022-05-05 07:00:00,2023-01-17 22:43:09,8,0,39,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN


In [19]:
customer_loan_df.shape

(14454, 17)

In [20]:
customer_loan_df.target.unique()

array([0, 1])

In [21]:
customer_loan_df[customer_loan_df["target"] == 1]


,customer_id,loan_id,ACC_CREATION_DATETIME,APPLICATION_DATETIME,LOAN_ORIGINATION_DATETIME,max_days_late,target,account_to_application_days,n_sf_apps,first_app_date,last_app_date,n_bnpl_apps,n_bnpl_approved_apps,first_bnpl_app_date,last_bnpl_app_date,n_inquiries_l3m,n_inquiries_l6m
13,1881,14,2021-10-01 23:29:27,2022-05-05 07:00:00,2022-07-19 19:30:28,55,1,215,5.0,2022-02-13,2022-03-03,2.0,0.0,2022-03-25 05:17:17.377,2022-03-04 02:06:40.177,0.0,0.0
15,2888,16,2022-01-06 18:49:26,2022-05-05 07:00:00,2022-08-19 17:30:59,63,1,118,2.0,2022-01-06,2022-03-21,1.0,1.0,2022-03-22 01:09:37.569,2022-03-22 01:09:37.569,0.0,17.0
19,1148,20,2021-08-14 20:18:11,2022-05-05 07:00:00,2022-09-14 10:01:41,49,1,263,5.0,2021-11-14,2021-08-18,1.0,0.0,2022-04-23 03:31:00.402,2022-04-23 03:31:00.402,0.0,0.0
20,2092,21,2021-10-16 03:56:50,2022-05-05 07:00:00,2022-07-10 16:04:44,70,1,201,5.0,2021-11-04,2021-10-16,NaN,NaN,NaT,NaT,0.0,0.0
34,3420,35,2022-02-11 13:21:38,2022-05-05 07:00:00,2022-09-19 12:46:37,49,1,82,6.0,2022-02-17,2022-02-11,2.0,0.0,2022-03-31 13:27:22.991,2022-03-06 07:50:20.253,76.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14383,13669,14384,2023-03-07 16:10:43,2023-04-25 07:00:00,2023-04-27 21:22:14,41,1,48,2.0,2023-03-07,2023-03-30,1.0,0.0,2023-03-07 21:11:51.882,2023-03-07 21:11:51.882,91.0,91.0
14385,2561,14386,2021-11-30 14:33:55,2023-04-25 07:00:00,2023-04-28 18:12:14,63,1,510,2.0,2022-10-13,2022-01-27,1.0,0.0,2022-04-13 02:37:57.994,2022-04-13 02:37:57.994,0.0,0.0
14391,9535,14392,2022-09-28 13:01:05,2023-04-26 07:00:00,2023-04-28 12:21:23,70,1,209,2.0,2022-09-30,2022-12-15,1.0,0.0,2022-10-05 05:16:47.042,2022-10-05 05:16:47.042,0.0,0.0
14395,855,14396,2021-07-10 15:41:15,2023-04-26 07:00:00,2023-04-28 19:20:18,69,1,654,NaN,NaT,NaT,1.0,0.0,2023-01-28 02:41:10.401,2023-01-28 02:41:10.401,13.0,13.0


In [22]:
credit_report_df.head()

,customer_id,INQUIRY_TIME,CDC_INQUIRY_ID,INQUIRY_DATE,PREVENTION_KEY,CURRENCY,MAX_CREDIT,CREDIT_LIMIT,PAYMENT_AMOUNT,UPDATE_DATE,...,ACCOUNT_TYPE,RESPONSABILITY_TYPE,TOTAL_PAYMENTS,DELAYED_PAYMENTS,CURRENT_PAYMENT,WORST_DELAY,TOTAL_REPORTED_PAYMENTS,CURRENT_BALANCE,BALANCE_DUE,BALANCE_DUE_WORST_DELAY
0,4223,2022-04-01 00:32:36.689000+00:00,710278-27993a6e-2885-48d4-a753-9249e7022af1,2022-04-01,SIN OBSERVACION,PESOS MEXICANOS,1244.0,9900.0,1244.0,2022-02-28,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),12.0,7,13,13.0,0,1244.0,1244.0,1244.0
1,4223,2022-04-01 00:32:36.689000+00:00,710278-27993a6e-2885-48d4-a753-9249e7022af1,2022-04-01,SIN OBSERVACION,PESOS MEXICANOS,1722.0,9900.0,1722.0,2022-02-28,...,REVOLVENTE,INDIVIDUAL (TITULAR),0.0,3,13,13.0,0,1722.0,1722.0,1722.0
2,4223,2022-04-01 00:32:36.689000+00:00,710278-27993a6e-2885-48d4-a753-9249e7022af1,2022-04-01,SIN OBSERVACION,PESOS MEXICANOS,6346.0,0.0,482.0,2022-03-28,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),16.0,0,V,0.0,0,943.0,0.0,0.0
3,3490,2022-02-15 02:30:22.086000+00:00,622857-6b4e9d95-7491-40c3-bccd-442de7f94c58,2022-02-15,SIN OBSERVACION,PESOS MEXICANOS,11600.0,0.0,232.0,2022-02-01,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),100.0,4,01,1.0,0,6185.0,116.0,116.0
4,6486,2022-06-25 01:57:14.868000+00:00,875073-46a5f149-19db-4193-b92a-b8f41fb3e896,2022-06-25,SIN OBSERVACION,PESOS MEXICANOS,2452.0,16800.0,2452.0,2021-09-30,...,REVOLVENTE,INDIVIDUAL (TITULAR),0.0,19,13,13.0,0,2452.0,2452.0,2452.0


In [23]:
customer_loan_df[customer_loan_df["customer_id"]==1881]

,customer_id,loan_id,ACC_CREATION_DATETIME,APPLICATION_DATETIME,LOAN_ORIGINATION_DATETIME,max_days_late,target,account_to_application_days,n_sf_apps,first_app_date,last_app_date,n_bnpl_apps,n_bnpl_approved_apps,first_bnpl_app_date,last_bnpl_app_date,n_inquiries_l3m,n_inquiries_l6m
13,1881,14,2021-10-01 23:29:27,2022-05-05 07:00:00,2022-07-19 19:30:28,55,1,215,5.0,2022-02-13,2022-03-03,2.0,0.0,2022-03-25 05:17:17.377,2022-03-04 02:06:40.177,0.0,0.0


In [24]:
credit_report_df[credit_report_df["customer_id"]==1881]

,customer_id,INQUIRY_TIME,CDC_INQUIRY_ID,INQUIRY_DATE,PREVENTION_KEY,CURRENCY,MAX_CREDIT,CREDIT_LIMIT,PAYMENT_AMOUNT,UPDATE_DATE,...,ACCOUNT_TYPE,RESPONSABILITY_TYPE,TOTAL_PAYMENTS,DELAYED_PAYMENTS,CURRENT_PAYMENT,WORST_DELAY,TOTAL_REPORTED_PAYMENTS,CURRENT_BALANCE,BALANCE_DUE,BALANCE_DUE_WORST_DELAY
6796,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,CUENTA QUE NO ESTA AL CORRIENTE VENDIDA O CEDI...,PESOS MEXICANOS,7428.0,9500.0,0.0,2018-09-08,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),NaN,9,06,6.0,None,0.0,14441.0,0.0
6797,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,SIN OBSERVACION,PESOS MEXICANOS,3000.0,3000.0,3556.0,2020-10-31,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),1.0,0,V,0.0,0,3556.0,0.0,0.0
6798,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,SIN OBSERVACION,PESOS MEXICANOS,5659.0,0.0,0.0,2021-02-20,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),4.0,0,V,0.0,None,0.0,0.0,0.0
6799,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,SIN OBSERVACION,PESOS MEXICANOS,3386.0,0.0,0.0,2021-05-07,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),6.0,3,V,1.0,None,0.0,0.0,0.0
6800,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,CUENTA CANCELADA O CERRADA,PESOS MEXICANOS,1500.0,1500.0,0.0,2021-07-31,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),1.0,16,V,23.0,0,0.0,0.0,3941.0
6801,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,SIN OBSERVACION,PESOS MEXICANOS,1500.0,1500.0,5382.0,2021-08-30,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),1.0,2,36,36.0,0,5382.0,5382.0,5382.0
6802,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,CUENTA EN COBRANZA,PESOS MEXICANOS,13034.0,13034.0,13034.0,2021-08-31,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),1.0,7,07,7.0,0,13034.0,13034.0,13034.0
6803,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,SIN OBSERVACION,PESOS MEXICANOS,13253.0,16800.0,13253.0,2021-08-31,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),12.0,2,13,13.0,0,13253.0,13253.0,13253.0
6804,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,SIN OBSERVACION,PESOS MEXICANOS,6818.0,16800.0,6818.0,2021-08-31,...,PAGOS FIJOS,INDIVIDUAL (TITULAR),12.0,2,13,13.0,0,6818.0,6818.0,6818.0
6805,1881,2021-10-02 20:13:53.462000+00:00,333627-ce4a0d8b-b5a0-4a58-a5b3-9f5153828cf5,2021-10-02,SIN OBSERVACION,PESOS MEXICANOS,83.0,16800.0,83.0,2021-08-31,...,REVOLVENTE,INDIVIDUAL (TITULAR),0.0,2,13,13.0,0,83.0,83.0,83.0


In [25]:
credit_report_df['PREVENTION_KEY'].unique()

array(['SIN OBSERVACION', 'CUENTA CANCELADA O CERRADA',
       'CONVENIO DE FINIQUITO O PAGO MENOR',
       'CUENTA REESTRUCTURADA SIN PAGO MENOR POR MODIFICACION DE LA SITUACION DEL CLIENTE',
       'CUENTA EN COBRANZA', 'CUENTA QUE CAUSA QUEBRANTO',
       'CUENTA QUE NO ESTA AL CORRIENTE VENDIDA O CEDIDA A UN NO USUARIO DE UNA SOCIEDAD',
       'CUENTA QUE NO ESTA AL CORRIENTE VENDIDA O CEDIDA A UN USUARIO DE UNA SOCIEDAD',
       'CUENTA INACTIVA',
       'CUENTA CERRADA QUE ESTUVO EN COBRANZA Y FUE PAGADA SIN CAUSAR QUEBRANTO',
       None,
       'EJECUCION DE GARANTIA PRENDARIA O FIDUCIARIA EN PAGO POR CREDITO',
       'PEGO MENOR POR PROGRAMA INSTITUCIONAL DE GOBIERNO',
       'TARJETA DE CREDITO EXTRAVIADA O ROBADA',
       'DEMANDA POR EL OTORGANTE', 'EN LOCALIZACION',
       'CONVENIO Y DISMINUCION DE PAGO', 'PRORROGA POR PERDIDA DE EMPLEO',
       'CARTERA AL CORRIENTE VENDIDA O CEDIDA A UN USUARIO DE UNA SOCIEDAD',
       'DACION EN PAGO O RENTA', 'ADJUDICACION O APLICACIO

In [27]:
credit_report_df.shape, customer_loan_df.shape

((287356, 29), (14454, 17))

In [29]:
customer_loan_df["n_bnpl_approved_apps"]

0        0.0
1        NaN
2        NaN
3        0.0
4        0.0
        ... 
14449    NaN
14450    NaN
14451    NaN
14452    0.0
14453    0.0
Name: n_bnpl_approved_apps, Length: 14454, dtype: float64